# <center>决策树(ID3)</center>

## 模型
假设当前节点数据集D1，两个特征A1,A2,信息增益
$$g(D1,A1)=H(D1)-H(D1|A1)$$
$$g(D1,A2)=H(D1)-H(D1|A2)$$
如果g(D1,A1)>g(D1,A2)则选特征A1，否则选特征A2

说明:g(D1,A1)>g(D1,A2),则H(D1|A1)<H(D1|A2),熵越小=不确定性越小=越确定，所以决策树越往底层越能确定分类

## 公式
$$H(D)=-\sum_{k=1}^{K} \frac{|C_{k}|}{|D|} \log _{2} \frac{|C_{k}|}{|D|}$$

$$H(D | A)=\sum_{i=1}^{n} \frac{|D_{i}|}{|D|} H(D_{i})$$

In [44]:
from collections import Counter

import numpy as np


class Node:
    def __init__(self,leaf,feature_index=None,label=None):
        self.leaf=leaf
        self.feature_index=feature_index
        self.label=label
        self.children={}

    def add_child(self,feature_value,node):
        self.children[feature_value]=node

    def predict(self,x):
        if self.leaf:
            return self.label
        return self.children[x[self.feature_index]].predict(x)

# ID3
class DecisionTree:
    def __init__(self,epsilon=0.1):
        self.epsilon=epsilon
        self._tree=None

    def fit(self,X,y):
        # 合在一起便于一起拆分
        data=np.hstack((X,y.reshape((-1,1))))
        self._tree=self._build_tree(data,np.arange(X.shape[1]))

    def predict(self,x):
        return self._tree.predict(x)

    # 只用于验证模型的正确性
    def score(self, X,y):
        count=0
        for x,yi in zip(X, y):
            if self.predict(x)==yi:
                count+=1
        return count/float(len(X))

    def _build_tree(self,data,features_index):
        y = data[:, -1]
        unique_labels=np.unique(y)

        if len(unique_labels)==1:
            return Node(leaf=True,label=unique_labels[0])

        if len(features_index)==0:
            return Node(leaf=True,label=self._most(y))

        max_info_gain_feature_index,max_info_gain=self._max_info_gain(data, features_index)
        if max_info_gain<self.epsilon:
            return Node(leaf=True,label=self._most(y))

        node=Node(leaf=False,feature_index=max_info_gain_feature_index)
        features_index_sub=np.delete(features_index,max_info_gain_feature_index)
        for feature_value,data_sub in self._groupBy(data, max_info_gain_feature_index):
            node.add_child(feature_value,self._build_tree(data_sub,features_index_sub))
        return node

    def _most(self, y):
        counter=Counter(y)
        return counter.most_common(1)[0]

    def _max_info_gain(self, data, features_index):
        y=data[:,-1]
        H_D=self._entropy(y)
        options=[]
        for feature_index in features_index:
            H_DA=self._cond_entropy(data,feature_index)
            options.append((feature_index,H_D-H_DA))
        return max(options,key=lambda x:x[1])

    # return [("a",[["a","b","y"])]
    def _groupBy(self, data, column):
        res=[]
        data=data[data[:, column].argsort()]
        value_pre=None
        data_sub=None
        for i in range(len(data)+1):
            if  i ==len(data) or (i != 0 and data[i, column] != value_pre):
                res.append((value_pre,data_sub))
                data_sub=None
            if i!=len(data):
                row=data[i,:]
                if data_sub is None:
                    data_sub=row.reshape(1,-1)
                else:
                    data_sub=np.vstack((data_sub,row))
                value_pre=row[column]
        return res

    # H(D)
    def _entropy(self,y):
        length = len(y)
        label_count = {}
        for i in range(length):
            label = y[i]
            if label not in label_count:
                label_count[label] = 0
            label_count[label] += 1
        return -np.sum([(v / float(length) * np.log2(v / length)) for v in label_count.values()])


    # H(D|A)
    def _cond_entropy(self,data,feature_index):
        return sum([(len(Di)/float(len(data)))*self._entropy(Di[:,-1]) for _,Di in self._groupBy(data,feature_index)])


def create_data():
     datasets =np.array([['青年', '否', '否', '一般', '否'],
               ['青年', '否', '否', '好', '否'],
               ['青年', '是', '否', '好', '是'],
               ['青年', '是', '是', '一般', '是'],
               ['青年', '否', '否', '一般', '否'],
               ['中年', '否', '否', '一般', '否'],
               ['中年', '否', '否', '好', '否'],
               ['中年', '是', '是', '好', '是'],
               ['中年', '否', '是', '非常好', '是'],
               ['中年', '否', '是', '非常好', '是'],
               ['老年', '否', '是', '非常好', '是'],
               ['老年', '否', '是', '好', '是'],
               ['老年', '是', '否', '好', '是'],
               ['老年', '是', '否', '非常好', '是'],
               ['老年', '否', '否', '一般', '否'],
               ])
     return datasets[:,:-1],datasets[:,-1]

X,y=create_data()

model=DecisionTree()
model.fit(X,y)
print(f"score:{model.score(X,y)}")

score:1.0
